## Data preparation with PANDAS

In [77]:
import pandas as pd


# Power plant conditions
p_conditions = pd.DataFrame({"Power plant":           ["Ahlen", "Fjället",  "Forsen",   "Kärret"],
                           "Initial reservoir level": [5800e6,  1000e6,     20e6,       13e6    ],
                           "Maximum reservoir level": [7160e6,  1675e6,     27e6,       13e6    ],
                           "Minimum reservoir level": [5800e6,  1000e6,     10e6,       6e6     ],
                           "Discharge capacity":      [540,     135,        975,        680     ],
                           "Power conversion":        [0.52,    1.17,       0.29,       0.05    ],
                           "Maximum spillage":        [820,     930,        360,        400     ],
                           "Local flow":              [177,     28,         8,          29      ],
                          })

# Time conditions
t_conditions = pd.DataFrame({"Time": range(1,12),
                             "Cost1": [45, 55, 80, 80, 110, 110, 80, 30, 70, 0, 0],
                             "Cost2": [45, 55, 95, 80, 80, 130, 130, 60, 95, 0, 0],
                             "Cost3": [45, 55, 120, 90, 140, 105, 80, 90, 120, 0, 0]})

tt_condtions = pd.melt(t_conditions, id_vars=["Time"], var_name="Cost", value_name="Value") # meh men orkar inte

# Flow conditions (Connections between the power plants)
f_conditions = pd.DataFrame({"From": ["Ahlen",  "Fjället",  "Forsen"],
                             "To":   ["Forsen", "Forsen",   "Kärret"],
                             "Time": [2,        1,          1]
                            })


## GAMSPy FTW

### Initializing all our Parameters and Varaibales etc

In [78]:

from gamspy import Container, Set, Variable, Parameter, Equation, Sum, Model, Sense, Alias


m = Container()

t = Set(m, name="t", description="time in hours", records=t_conditions['Time']) # time at begining of hour 1, 2, 3, ...
p = Set(m, name="p", description="Power plant", records=p_conditions['Power plant'])
s = Set(m, name="s", description="Scenario", records=tt_condtions['Cost'].unique())

# Create alias for set p
p_up = Alias(m, name="p_up", alias_with=p)

# Parameter and Variables definitions


delay = Parameter(m, name="delay", domain=[p,p], description="Time delay for upstream plants", records=f_conditions[['From', 'To', 'Time']])

prices = Parameter(m, name="prices", domain=[t,s], description="Prices (MWh) at different hours", records=tt_condtions[['Time', 'Cost', 'Value']])

reservoir_init = Parameter(m, name="reservoir_init", domain=p, description="Initial reservoir level", records=p_conditions[['Power plant', 'Initial reservoir level']])
reservoir_max = Parameter(m, name="reservoir_max", domain=p, description="Maximum reservoir level", records=p_conditions[['Power plant', 'Maximum reservoir level']])
reservoir_min = Parameter(m, name="reservoir_min", domain=p, description="Minimum reservoir level", records=p_conditions[['Power plant', 'Minimum reservoir level']])
discharge_max = Parameter(m, name="discharge_max", domain=p, description="Discharge capacity", records=p_conditions[['Power plant', 'Discharge capacity']])
power_conversion = Parameter(m, name="power_conversion", domain=p, description="Power conversion", records=p_conditions[['Power plant', 'Power conversion']])
spillage_max = Parameter(m, name="spillage_max", domain=p, description="Maximum spillage", records=p_conditions[['Power plant', 'Maximum spillage']])
local_flow = Parameter(m, name="local_flow", domain=p, description="Local flow", records=p_conditions[['Power plant', 'Local flow']])

# Variables
discharge = Variable(m, name="discharge", type="positive", domain=[t,p,s], description="Discharge rate at each power plant at each time")
spillage = Variable(m, name="spillage", type="positive", domain=[t,p,s], description="Spillage rate at each power plant at each time")
reservoir_level = Variable(m, name="reservoir_level", domain=[t,p,s], description="Reservoir level at each power plant at each time")
potential_volume = Variable(m, name="potential_volume", domain=[p,s], description="Potential volume at each power plant at last time (T=10)")

### Equations and condtions

In [79]:
# Discharge criteria
discharge.up[t,p,s] = discharge_max[p]

# Spillage criteria
spillage.up[t,p,s] = spillage_max[p]

# Reservoir level criteria
reservoir_level.lo[t,p,s].where[t.ord > 1] = reservoir_min[p]
reservoir_level.up[t,p,s].where[t.ord > 1] = reservoir_max[p]
reservoir_level.fx[t,p,s].where[t.first] = reservoir_init[p] # Initial reservoir level should be set to reservoir initial level

# Strategic lock criteria 
strategic_lock = Equation(m, name="strategic_lock", domain=[t,p,s], description="Strategic lock at the first two hours")
strategic_lock[t,p,s].where[(t.ord < 3) & (s.ord < 3)] = discharge[t,p,s] == discharge[t,p,s+1]

# Potential volume criteria
potential = Equation(m, name="potential", domain=[p,s], description="Potential volume at each power plant based on the volume at last time (T=10) and power plant above")
potential[p,s] = potential_volume[p,s] == Sum(p_up.where[delay[p_up,p]>0], potential_volume[p_up,s]) + reservoir_level["10",p,s] - reservoir_min[p]

# Single reservoir equation for all plants
reservoirs = Equation(m, name="reservoirs", domain=[t,p,s], description="Reservoir level at power plant (p) at different hours (t)")
reservoirs[t,p,s].where[t.ord > 1] = reservoir_level[t,p,s] == reservoir_level[t.lag(1),p,s] + 3600 * (
        # Upstream inflows
        Sum(p_up.where[delay[p_up,p]>0], 
            discharge[t.lag(delay[p_up, p]), p_up, s] + spillage[t.lag(delay[p_up, p]), p_up, s]
        )

        # Local inflow
        + local_flow[p]

        # Outflows
        - discharge[t.lag(1),p,s]
        - spillage[t.lag(1),p,s]
    )

### Obejctive

In [80]:
obj = Sum((t, p, s), 1/3*prices[t,s]*3600*power_conversion[p]*discharge[t,p,s]) + Sum((p,s), 1/3*95*power_conversion[p]*potential_volume[p,s])

## Solution

In [81]:
flow = Model(m, name="flow", equations=m.getEquations(), objective=obj, problem="LP", sense=Sense.MAX)
flow.solve(solver="CPLEX")

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,1123245480.00002,149,385,LP,CPLEX,0.003


1406579020	 1040125726.66679

In [82]:
from IPython.display import HTML

def horizontal(dfs):
    html = '<div style="display:flex">'
    for df in dfs:
        html += '<div style="margin-right: 32px">'
        html += df.to_html()
        html += '</div>'
    html += '</div>'
    display(HTML(html))

In [83]:
# Display all discharge records for a specific power plant and scenario
reservoir_level.records.loc[(reservoir_level.records['s'] == 'Cost1')].head(8)

,t,p,s,level,marginal,lower,upper,scale
0,1,Ahlen,Cost1,5.800000e+09,29.833333,5.800000e+09,5.800000e+09,1.0
3,1,Fjället,Cost1,1.000000e+09,53.666667,1.000000e+09,1.000000e+09,1.0
6,1,Forsen,Cost1,2.000000e+07,10.766667,2.000000e+07,2.000000e+07,1.0
9,1,Kärret,Cost1,1.300000e+07,0.833333,1.300000e+07,1.300000e+07,1.0
12,2,Ahlen,Cost1,5.800637e+09,0.000000,5.800000e+09,7.160000e+09,1.0
15,2,Fjället,Cost1,1.000101e+09,0.000000,1.000000e+09,1.675000e+09,1.0
18,2,Forsen,Cost1,2.002880e+07,0.000000,1.000000e+07,2.700000e+07,1.0
21,2,Kärret,Cost1,1.300000e+07,0.500000,6.000000e+06,1.300000e+07,1.0


In [84]:
reservoir_level.records.loc[(reservoir_level.records['s'] == 'Cost2')].head(8)

,t,p,s,level,marginal,lower,upper,scale
1,1,Ahlen,Cost2,5.800000e+09,27.233333,5.800000e+09,5.800000e+09,1.0
4,1,Fjället,Cost2,1.000000e+09,61.466667,1.000000e+09,1.000000e+09,1.0
7,1,Forsen,Cost2,2.000000e+07,10.766667,2.000000e+07,2.000000e+07,1.0
10,1,Kärret,Cost2,1.300000e+07,1.416667,1.300000e+07,1.300000e+07,1.0
13,2,Ahlen,Cost2,5.800637e+09,0.000000,5.800000e+09,7.160000e+09,1.0
16,2,Fjället,Cost2,1.000101e+09,0.000000,1.000000e+09,1.675000e+09,1.0
19,2,Forsen,Cost2,2.002880e+07,0.000000,1.000000e+07,2.700000e+07,1.0
22,2,Kärret,Cost2,1.300000e+07,0.000000,6.000000e+06,1.300000e+07,1.0


In [85]:
reservoir_level.records.loc[(reservoir_level.records['s'] == 'Cost3')].head(8)

,t,p,s,level,marginal,lower,upper,scale
2,1,Ahlen,Cost3,5.800000e+09,31.566667,5.800000e+09,5.800000e+09,1.0
5,1,Fjället,Cost3,1.000000e+09,57.566667,1.000000e+09,1.000000e+09,1.0
8,1,Forsen,Cost3,2.000000e+07,10.766667,2.000000e+07,2.000000e+07,1.0
11,1,Kärret,Cost3,1.300000e+07,-0.000000,1.300000e+07,1.300000e+07,1.0
14,2,Ahlen,Cost3,5.800637e+09,0.000000,5.800000e+09,7.160000e+09,1.0
17,2,Fjället,Cost3,1.000101e+09,0.000000,1.000000e+09,1.675000e+09,1.0
20,2,Forsen,Cost3,2.002880e+07,0.000000,1.000000e+07,2.700000e+07,1.0
23,2,Kärret,Cost3,1.300000e+07,0.000000,6.000000e+06,1.300000e+07,1.0


In [87]:
discharge.records.head(24)

,t,p,s,level,marginal,lower,upper,scale
0,1,Ahlen,Cost1,0.0,0.0,0.0,540.0,1.0
1,1,Ahlen,Cost2,0.0,-118560.0,0.0,540.0,1.0
2,1,Ahlen,Cost3,0.0,0.0,0.0,540.0,1.0
3,1,Fjället,Cost1,0.0,0.0,0.0,135.0,1.0
4,1,Fjället,Cost2,0.0,-315900.0,0.0,135.0,1.0
5,1,Fjället,Cost3,0.0,0.0,0.0,135.0,1.0
6,1,Forsen,Cost1,0.0,0.0,0.0,975.0,1.0
7,1,Forsen,Cost2,0.0,-61200.0,0.0,975.0,1.0
8,1,Forsen,Cost3,0.0,0.0,0.0,975.0,1.0
9,1,Kärret,Cost1,29.0,0.0,0.0,680.0,1.0
